# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0  port-aux-francais -49.3500   70.2167    277.69        62   
1        1     margaret river -33.9500  115.0667    294.77        32   
2        2          grytviken -54.2811  -36.5092    271.31        77   
3        3               kone -21.0595  164.8658    295.94        94   
4        4        yellowknife  62.4560 -114.3525    271.71        94   

   Cloudiness  Wind Speed Country        Date  
0          95       12.70      TF  1713943968  
1         100        4.19      AU  1713943969  
2          35       10.23      GS  1713943971  
3         100        1.89      NC  1713943972  
4          75        3.60      CA  1713943945

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City")

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.count()

# Drop any rows with null values
clean_city_data_df = city_data_df.dropna(how='any')

# Display sample data
clean_city_data_df

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0          0  port-aux-francais -49.3500   70.2167    277.69        62   
1          1     margaret river -33.9500  115.0667    294.77        32   
2          2          grytviken -54.2811  -36.5092    271.31        77   
3          3               kone -21.0595  164.8658    295.94        94   
4          4        yellowknife  62.4560 -114.3525    271.71        94   
..       ...                ...      ...       ...       ...       ...   
591      591     merizo village  13.2658  144.6691    303.47        82   
592      592              tabuk  17.4189  121.4443    307.80        47   
593      593        butterworth   5.3991  100.3638    306.17        66   
594      594        broken hill -31.9500  141.4333    290.44        40   
595      595       wladyslawowo  54.7909   18.4009    276.45        70   

     Cloudiness  Wind Speed Country        Date  
0            95       12.70      TF  1713943968  
1           100        4.19      AU  1713943969  
2            35       10.23      GS  1713943971  
3           100        1.89      NC  1713943972  
4            75        3.60      CA  1713943945  
..          ...         ...     ...         ...  
591          85        6.83      GU  1714025542  
592          59        2.59      PH  1714025543  
593          40        2.57      MY  1714025545  
594          52        7.17      AU  1714025546  
595          85        2.76      PL  1714025547  

[593 rows x 10 columns]

##### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
reduced_city_data_df = clean_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_data_df = pd.DataFrame({"City": reduced_city_data_df["City"], "Country": reduced_city_data_df["Country"], "Lat": reduced_city_data_df["Lat"], 
                              "Lng": reduced_city_data_df["Lng"], "Humidity": reduced_city_data_df["Humidity"], "Hotel Name": ""})

# Display sample data
hotel_data_df

City Country      Lat       Lng  Humidity Hotel Name
0    port-aux-francais      TF -49.3500   70.2167        62           
1       margaret river      AU -33.9500  115.0667        32           
2            grytviken      GS -54.2811  -36.5092        77           
3                 kone      NC -21.0595  164.8658        94           
4          yellowknife      CA  62.4560 -114.3525        94           
..                 ...     ...      ...       ...       ...        ...
591     merizo village      GU  13.2658  144.6691        82           
592              tabuk      PH  17.4189  121.4443        47           
593        butterworth      MY   5.3991  100.3638        66           
594        broken hill      AU -31.9500  141.4333        40           
595       wladyslawowo      PL  54.7909   18.4009        70           

[593 rows x 6 columns]

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_data_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_data_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_data_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_data_df.loc[index, 'City']} - nearest hotel: {hotel_data_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_data_df

Starting hotel search
port-aux-francais - nearest hotel: Keravel
margaret river - nearest hotel: Margaret River Hotel
grytviken - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
yellowknife - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
mamuju - nearest hotel: Hotel d'Maleo
goundam - nearest hotel: No hotel found
rikaze - nearest hotel: Gesar Hotel
susuman - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
usogorsk - nearest hotel: Единственная
waitangi - nearest hotel: Hotel Chathams
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
aasiaat - nearest hotel: SØMA
vorgashor - nearest hotel: София
almenara - nearest hotel: No hotel found
villa regina - nearest hotel: Nuevo Cheo
gizo - nearest hotel: Gizo Hotel
fushun - nearest hotel: 友谊宾馆
aneho - nearest hotel: Mater Christi
blackmans bay - nearest hotel: Villa Howden
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
m

City Country      Lat       Lng  Humidity  \
0    port-aux-francais      TF -49.3500   70.2167        62   
1       margaret river      AU -33.9500  115.0667        32   
2            grytviken      GS -54.2811  -36.5092        77   
3                 kone      NC -21.0595  164.8658        94   
4          yellowknife      CA  62.4560 -114.3525        94   
..                 ...     ...      ...       ...       ...   
591     merizo village      GU  13.2658  144.6691        82   
592              tabuk      PH  17.4189  121.4443        47   
593        butterworth      MY   5.3991  100.3638        66   
594        broken hill      AU -31.9500  141.4333        40   
595       wladyslawowo      PL  54.7909   18.4009        70   

               Hotel Name  
0                 Keravel  
1    Margaret River Hotel  
2          No hotel found  
3       Pacifik Appartels  
4          No hotel found  
..                    ...  
591        No hotel found  
592        Davidson Hotel  
593        No hotel found  
594           Ibis Styles  
595               Titanic  

[593 rows x 6 columns]

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)